# Анализ сервиса вопросов и ответов по программированию

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id=1 AND (score>300 OR favorites_count>=100 );

Cколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
WITH posts AS
(SELECT creation_date::DATE,
        COUNT(id) AS post
FROM stackoverflow.posts        
WHERE post_type_id=1 AND (creation_date::DATE BETWEEN '2008-11-01' AND '2008-11-18')
GROUP BY creation_date::DATE)
SELECT ROUND(AVG(post))
FROM posts;

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
LEFT JOIN stackoverflow.badges AS b ON b.user_id = u.id
WHERE b.creation_date::DATE = u.creation_date::DATE;

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users AS u 
JOIN stackoverflow.posts AS p ON u.id=p.user_id
JOIN stackoverflow.votes AS v ON p.id=v.post_id
WHERE u.display_name = 'Joel Coehoorn';

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
WITH vvt AS
(SELECT *
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON vt.id=v.vote_type_id
WHERE vt.name = 'Close')
SELECT user_id,
       COUNT(user_id) AS total
FROM vvt 
GROUP BY user_id
ORDER BY total DESC, user_id DESC
LIMIT 10;


Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
WITH badges AS
(SELECT user_id,
       COUNT(id) AS total
FROM stackoverflow.badges
WHERE creation_date::DATE BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id)
SELECT *,
       DENSE_RANK() OVER (ORDER BY total DESC)
FROM badges
ORDER BY total DESC, user_id
LIMIT 10;

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
заголовок поста;
идентификатор пользователя;
число очков поста;
среднее число очков пользователя за пост, округлённое до целого числа.
Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
WITH posts AS
(SELECT user_id,
       ROUND(AVG(score)) AS mid
FROM stackoverflow.posts
WHERE score != 0 AND title IS NOT NULL
GROUP BY user_id)
SELECT posts1.title,
       posts1.user_id,
       posts1.score,
       posts.mid
FROM stackoverflow.posts AS posts1
JOIN posts ON posts1.user_id = posts.user_id
WHERE posts1.score != 0 AND posts1.title IS NOT NULL;

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список

In [ ]:
WITH badges AS
(SELECT user_id,
       COUNT(user_id) AS total
FROM stackoverflow.badges
GROUP BY user_id
HAVING COUNT(user_id) > 1000)
SELECT posts.title
FROM stackoverflow.posts AS posts
JOIN badges ON badges.user_id=posts.user_id
WHERE title IS NOT NULL;

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
пользователям с числом просмотров меньше 100 — группу 3.
Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       CASE
          WHEN views >=350 THEN 1
          WHEN views <350 AND views >=100 THEN 2
          WHEN views <100 THEN 3
       END
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views !=0;

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH users AS
(SELECT id,
       views,
       (CASE
          WHEN views >=350 THEN 1
          WHEN views <350 AND views >=100 THEN 2
          WHEN views <100 THEN 3
       END) AS lev
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views !=0),
users2 AS
(SELECT id,
       views,
       lev,
       MAX(views) OVER (PARTITION BY lev) AS mikc
FROM users)
SELECT users2.id,
       users2.views,
       users2.lev
FROM users2
WHERE users2.views=users2.mikc
ORDER BY views DESC, id;

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
номер дня;
число пользователей, зарегистрированных в этот день;
сумму пользователей с накоплением.

In [ ]:
WITH users AS
(SELECT *
FROM stackoverflow.users
WHERE creation_date BETWEEN '2008-11-01' AND '2008-12-01'),
users2 AS
(SELECT EXTRACT(DAY FROM creation_date) AS day,
        COUNT(id) AS summa
FROM users
GROUP BY EXTRACT(DAY FROM creation_date))
SELECT day,
       summa,
       SUM(summa) OVER (ORDER BY day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM users2
ORDER BY day;

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
идентификатор пользователя;
разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT DISTINCT posts.user_id,
       MIN(posts.creation_date) OVER (PARTITION BY posts.user_id) - users.creation_date 
FROM stackoverflow.posts AS posts
LEFT JOIN stackoverflow.users AS users ON posts.user_id=users.id
GROUP BY posts.user_id, posts.creation_date, users.creation_date;

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
WITH posts AS
(SELECT *
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-01-01' AND '2008-12-31')
SELECT DATE_TRUNC('month', creation_date)::DATE,
       SUM(views_count) 
FROM posts
GROUP BY DATE_TRUNC('month', creation_date)::DATE
ORDER BY SUM(views_count) DESC;

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id=u.id
JOIN stackoverflow.post_types AS pt ON pt.id=p.post_type_id
WHERE p.creation_date::DATE BETWEEN u.creation_date::DATE AND (u.creation_date::DATE + INTERVAL '1 month') 
      AND pt.TYPE LIKE '%Answer%'
GROUP BY u.display_name
HAVING COUNT(p.id) > 100
ORDER BY u.display_name;


Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH id_norm AS
(SELECT DISTINCT u.id
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
WHERE CAST(DATE_TRUNC('month', u.creation_date) AS DATE) = '2008-09-01'
AND CAST(DATE_TRUNC('month', p.creation_date) AS DATE) = '2008-12-01')
SELECT CAST(DATE_TRUNC('month', p.creation_date) AS DATE) AS months,
       COUNT(p.id) AS total_posts
FROM stackoverflow.posts AS p
WHERE CAST(DATE_TRUNC('year', p.creation_date) AS DATE) = '2008-01-01'
      AND p.user_id IN (SELECT * FROM id_norm)
GROUP BY months
ORDER BY months DESC;

Используя данные о постах, выведите несколько полей:
идентификатор пользователя, который написал пост;
дата создания поста;
количество просмотров у текущего поста;
сумму просмотров постов автора с накоплением.
Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date;

Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
WITH norm AS
(WITH posts AS
(SELECT DISTINCT user_id,
       creation_date::date AS dat
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY creation_date, user_id)
SELECT COUNT(dat) OVER (PARTITION BY user_id) AS mid
FROM posts)
SELECT ROUND(AVG(mid))
FROM norm;


На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
номер месяца;
количество постов за месяц;
процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
WITH posts AS
(SELECT EXTRACT(MONTH FROM creation_date) AS month,
       COUNT(id) AS total
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY EXTRACT(MONTH FROM creation_date))
SELECT *,
       ROUND(((total::numeric / LAG(total) OVER (ORDER BY month)) - 1) * 100, 2)
FROM posts;

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
номер недели;
дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
SELECT DISTINCT EXTRACT(WEEK FROM creation_date::date),
       MAX(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date::date))
FROM stackoverflow.posts 
WHERE DATE_TRUNC('month', creation_date) = '2008-10-01'
    AND user_id IN (SELECT user_id
                                FROM (SELECT user_id,
                                       COUNT(id)
                                FROM stackoverflow.posts
                                GROUP BY 1
                                ORDER BY 2 DESC) AS a
                                LIMIT 1);